# Jupyter Lectures, First Year Project 2021

<font color="red">Instructor's version</font>

## Project 1, ITU Copenhagen

**Instructor: Michael Szell**

Course page: https://learnit.itu.dk/local/coursebase/view.php?ciid=590

This notebook wrangles and explores the data set from the project.

Contact: Michael Szell (misz@itu.dk)  
Created: 2021-01-11  
Last modified: 2021-02-15

<hr>

# Lecture 1: First data exploration

### Imports

In [ ]:
# First version
import numpy as np


# Add later as needed
import matplotlib
import matplotlib.pyplot as plt
import csv

import pandas as pd
import seaborn as sns
from scipy.stats import chi2_contingency

import json
import shapely
from shapely.geometry import Point, MultiPoint, LineString, MultiLineString, Polygon, MultiPolygon

import folium
from folium import plugins
from folium.plugins import HeatMap, MarkerCluster

In [ ]:
# Add later as needed
%run -i ../scripts/functions.py

### Constants

Constants are written all caps: https://www.python.org/dev/peps/pep-0008/#constants

In [ ]:
PATH = {}
PATH["data_raw"] = "../data/raw/"
PATH["data_interim"] = "../data/interim/"
PATH["data_processed"] = "../data/processed/"
PATH["data_external"] = "../data/external/"

FILENAME = {}
FILENAME["accidents"] = "Road Safety Data - Accidents 2019.csv"
FILENAME["casualties"] = "Road Safety Data - Casualties 2019.csv"
FILENAME["vehicles"] = "Road Safety Data- Vehicles 2019.csv" # Note the inconsistent file naming (no space before "-" here)

# Add later, lec 2
TABLENAMES = ["accidents", "casualties", "vehicles"]

### Load raw data

The data were downloaded from here on Jan 4th: https://data.gov.uk/dataset/road-accidents-safety-data  
That page was updated afterwards (Jan 8th), so local and online data may be inconsistent.

We first explore one data table, the accidents.

In [ ]:
# First version, just using the accident table
dataraw_acc = np.genfromtxt(PATH["data_raw"]+FILENAME["accidents"], delimiter=',', dtype=None, names=True, encoding='utf8')

# # Add later, end of lec 1
# dataraw = {}
# dataraw["accidents"] = np.genfromtxt(PATH["data_raw"]+FILENAME["accidents"], delimiter=',', dtype=None, names=True, encoding='utf-8-sig')
# dataraw["vehicles"] = np.genfromtxt(PATH["data_raw"]+FILENAME["vehicles"], delimiter=',', dtype=None, names=True, encoding='utf-8-sig')
# dataraw["casualties"] = np.genfromtxt(PATH["data_raw"]+FILENAME["casualties"], delimiter=',', dtype=None, names=True, encoding='utf-8-sig')

# Add later, in lec 2
dataraw = {}
for tablename in TABLENAMES:
    dataraw[tablename] = np.genfromtxt(PATH["data_raw"]+FILENAME[tablename], delimiter=',', dtype=None, names=True, encoding='utf-8-sig')

In [ ]:
# Add later, in lec 2
headerraw = {}
for tablename in TABLENAMES:
    headerraw[tablename] = list(dataraw[tablename].dtype.names)

It is always good to start with a "sneak preview":

In [ ]:
dataraw_acc[:5]

Reminder and documentation on structured arrays:  
https://numpy.org/devdocs/user/basics.rec.html

#### Insight: Mixed variable types

Accidents have mixed data types, including strings, floats, integers. Categorical variables are encoded as integers. The meaning of these categories can be looked up in `../references/variable lookup.xls`

Number of records

In [ ]:
dataraw_acc.shape

Number of fields

In [ ]:
len(dataraw_acc.dtype)

In [ ]:
dataraw_acc.dtype

**"Data in the wild" puzzle: Why is the first field "\ufeffAccident_Index" and not "Accident_Index"?**

Fields

In [ ]:
dataraw_acc.dtype.names

Solution: utf8 was the wrong encoding! The correct one seems to be utf-8-sig.

https://stackoverflow.com/questions/17912307/u-ufeff-in-python-string/17912811#17912811

<font color="red">Instructor: Go back to top and fix. Also make dict of tables. Explain code refactoring:</font>  
https://en.wikipedia.org/wiki/Code_refactoring

Homework: Explore the other two tables the same way.

<hr>

# Lecture 2: Command line wrangling and dealing with missing data

A faster way of getting basic insights into a new data set than by using numpy is by using command line tools.

Let's get a first overview using `head`. There are 3 data tables: Accidents, Casualties, and Vehicles.

In [ ]:
!head -n 6 "../data/raw/Road Safety Data - Accidents 2019.csv" 
!head -n 6 "../data/raw/Road Safety Data - Casualties 2019.csv"
!head -n 6 "../data/raw/Road Safety Data- Vehicles 2019.csv" 

### General insights

#### Link between data tables

Records between data tables are linked through their `Accident_Index`.

Looking at the first Accident_Index 2019010128300, we can see there seems to be a one-to-many relation between accident->casualty and accident->vehicle, meaning there can be multiple casualties and vehicles involved in one accident (makes sense).

https://en.wikipedia.org/wiki/One-to-many_(data_model)

#### Dimensions

Number of records

https://en.wikipedia.org/wiki/Wc_(Unix)

In [ ]:
!wc -l "../data/raw/Road Safety Data - Accidents 2019.csv" 
!wc -l "../data/raw/Road Safety Data - Casualties 2019.csv" 
!wc -l "../data/raw/Road Safety Data- Vehicles 2019.csv"

Number of fields (in first line)

https://www.geeksforgeeks.org/awk-command-unixlinux-examples/

In [ ]:
!head -1 "../data/raw/Road Safety Data - Accidents 2019.csv" | awk -F ',' '{print NF}'
!head -1 "../data/raw/Road Safety Data - Casualties 2019.csv" | awk -F ',' '{print NF}'
!head -1 "../data/raw/Road Safety Data- Vehicles 2019.csv" | awk -F ',' '{print NF}'

See and count all fields

https://en.wikipedia.org/wiki/Tr_(Unix)
https://en.wikipedia.org/wiki/Nl_(Unix)

In [ ]:
!head -1 "../data/raw/Road Safety Data - Accidents 2019.csv" | tr ',' '\n' | nl
!head -1 "../data/raw/Road Safety Data - Casualties 2019.csv" | tr ',' '\n' | nl
!head -1 "../data/raw/Road Safety Data- Vehicles 2019.csv" | tr ',' '\n' | nl

### Sanity checks

Has each record the same number of fields?

https://shapeshed.com/unix-uniq/  
https://www.putorius.net/uniq-command-linux.html

In [ ]:
!awk -F ',' '{print NF}' "../data/raw/Road Safety Data - Accidents 2019.csv" | sort | uniq -d
!awk -F ',' '{print NF}' "../data/raw/Road Safety Data - Casualties 2019.csv" | sort | uniq -d
!awk -F ',' '{print NF}' "../data/raw/Road Safety Data- Vehicles 2019.csv" | sort | uniq -d

How many duplicate lines are there? (If more than 0, there could be a problem)

In [ ]:
!sort "../data/raw/Road Safety Data - Accidents 2019.csv" | uniq -d  | wc -l
!sort "../data/raw/Road Safety Data - Casualties 2019.csv" | uniq -d | wc -l
!sort "../data/raw/Road Safety Data- Vehicles 2019.csv" | uniq -d  | wc -l

More advanced stuff with `awk`: https://datafix.com.au/BASHing/2020-05-20.html

## Dealing with missing data

Using a masked array:  
https://numpy.org/devdocs/reference/maskedarray.baseclass.html#numpy.ma.MaskedArray

<font color="red">Instructor: We want a nicer way to loop through table names. Let's go to beginning and refactor with TABLENAMES.</font>

In [ ]:
dataraw_masked = {}
for tablename in TABLENAMES:
    dataraw_masked[tablename] = np.genfromtxt(PATH["data_raw"]+FILENAME[tablename], delimiter=',', dtype=None, names=True, encoding='utf-8-sig', usemask=True)

In [ ]:
dataraw_masked["accidents"][:5]

In [ ]:
dataraw_masked["accidents"].mask[:5]

The first 5 rows seem complete. What about the rest?

In [ ]:
np.count_nonzero(dataraw_masked["accidents"].mask)

Oh oh, values are missing in 5776 rows! In which rows?

In [ ]:
rows_incomplete = np.where(dataraw_masked["accidents"].mask)[0]
print(rows_incomplete)

How many values in total?  
Which fields are missing?

In [ ]:
missingpositions = {}
missingvalues = 0 # Add later
missingconfigurations = set() # Add later
for rowpos in rows_incomplete:
    missingpositions_thisrow = list(np.where(list(dataraw_masked["accidents"].mask[rowpos]))[0])
    missingpositions[rowpos] = missingpositions_thisrow
    missingvalues += len(missingpositions_thisrow) # Add later
    missingconfigurations.add((tuple(missingpositions_thisrow))) # Add later

missingfieldnames = [np.array(headerraw["accidents"])[c] for c in [list(b) for b in missingconfigurations]] # Add later

In [ ]:
print(missingpositions) # Don't do this is you have more than a few 1000 rows or Jupyter might crash.

Summary of missing values:

In [ ]:
print("Incomplete rows: " + str(np.count_nonzero(dataraw_masked["accidents"].mask)))
print("Missing values: " + str(missingvalues))
print("\nMissing field configurations: " + str(missingconfigurations))
print("Missing field configurations (names): ")
for i in missingfieldnames:
    print(i)

<hr>

# Lecture 3: Visual data exploration, Connecting tables, Association test

## Visual exploratory data analysis ("Plot your data")

### Bar plots of categorical variables

In [ ]:
headerraw["accidents"]

In [ ]:
field_name = "Accident_Severity"
field_categories = {1: "Fatal", 2: "Serious", 3: "Slight"}

categories, counts = np.unique(dataraw["accidents"][field_name], return_counts=True)

fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1]) # left, bottom, width, height (range 0 to 1)
axes.bar(categories, counts, fc="gray") # fc is the face color

axes.set_xlabel("")
axes.set_ylabel('Count')
axes.set_title(field_name)

axes.set_xticks(list(field_categories.keys()))
axes.set_xticklabels(field_categories.values());

We were lucky because the categories 1,2,3 were "nice". But usually they aren't, so we need to explicitly map to integers:

In [ ]:
field_name = "Speed_limit"
field_categories = {20: "20 MPH", 30: "30 MPH", 40: "40 MPH", 50: "50 MPH", 60: "60 MPH", 70: "70 MPH", -1: "N/A"}

categories, counts = np.unique(dataraw["accidents"][field_name], return_counts=True)

fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1]) # left, bottom, width, height (range 0 to 1)
axes.bar(range(len(categories)), counts, fc="gray") # fc is the face color

axes.set_xlabel("")
axes.set_ylabel('Count')
axes.set_title(field_name)
fig.autofmt_xdate(rotation=45) # Add later

axes.set_xticks(range(len(categories)))
axes.set_xticklabels([field_categories[c] for c in categories]); # Changing this line is important!

Instead of copy-pasting code, let's write a function.

In [ ]:
# Add this line later for testing
%run -i ../scripts/functions.py

field_name = "Light_Conditions"
field_categories = {1: "Daylight", 4: "Darkness - lights lit", 5: "Darkness - lights unlit", 6: "Darkness - no lighting", 7: "Darkness - lighting unknown", -1: "Data missing or out of range"}

barplot(dataraw["accidents"], field_name, field_categories)

Typing the variable lookup manually is cumbersome. Can we read the excel directly?  
pandas can: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html

In [ ]:
import pandas as pd # Add to beginning
variable_lookup = pd.read_excel('../references/variable lookup.xls', sheet_name = None)

In [ ]:
print(variable_lookup.keys())
print("\n")
print(variable_lookup["Introduction"])

### Histograms of numerical variables

In [ ]:
headerraw["casualties"]

In [ ]:
fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1])
axes.hist(dataraw["casualties"]["Age_of_Casualty"]); # Add bins later

axes.set_ylabel('Counts')
axes.set_title("Age_of_Casualty");

In [ ]:
fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1])
axes.hist(dataraw["casualties"]["Age_of_Casualty"], bins = np.linspace(-1,121, 123)); # Add bins later

axes.set_ylabel('Counts')
axes.set_title("Age_of_Casualty");

In [ ]:
fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1])
axes.hist(dataraw["vehicles"]["Age_of_Driver"], bins = np.linspace(-1,121, 123)); # Try again with (0,121, 122)

axes.set_ylabel('Counts')
axes.set_title("Age_of_Driver");

### Categorical scatterplots

Scatterplots are good for relating two numerical variables. If we have one numerical versus one categorical variable, we can do a box plot. But could we also visualize all data points? Yes: https://seaborn.pydata.org/tutorial/categorical.html

In [ ]:
import seaborn as sns

In [ ]:
ind = (dataraw["casualties"]["Casualty_Type"] == 0) | (dataraw["casualties"]["Casualty_Type"] == 1) | (dataraw["casualties"]["Casualty_Type"] == 9) & (dataraw["casualties"]["Age_of_Casualty"] > -1)
ind[600:] = False
data_toplot = np.array([dataraw["casualties"]["Casualty_Type"][ind], dataraw["casualties"]["Age_of_Casualty"][ind]]).T

fig = sns.catplot(x='Casualty Type', y='Age of Casualty', data=pd.DataFrame(data_toplot, columns=['Casualty Type', 'Age of Casualty']), kind="swarm") # also show: violin
fig.set_xticklabels(["Pedestrian", "Cyclist", "Car occupant"]);

In [ ]:
ind = (dataraw["casualties"]["Casualty_Type"] == 0) | (dataraw["casualties"]["Casualty_Type"] == 1) | (dataraw["casualties"]["Casualty_Type"] == 9) & (dataraw["casualties"]["Age_of_Casualty"] > -1) & (dataraw["casualties"]["Sex_of_Casualty"] > -1)
ind[600:] = False
data_toplot = np.array([dataraw["casualties"]["Casualty_Type"][ind], dataraw["casualties"]["Age_of_Casualty"][ind], dataraw["casualties"]["Sex_of_Casualty"][ind]]).T

fig = sns.catplot(x='Casualty Type', y='Age of Casualty', hue='Sex of Casualty', data=pd.DataFrame(data_toplot, columns=['Casualty Type', 'Age of Casualty', 'Sex of Casualty']), kind="swarm") # also show: violin
fig.set_xticklabels(["Pedestrian", "Cyclist", "Car occupant"]);

## Connecting tables with np.isin()

**Question: How many babies and toddlers died or got injured on UK roads in June 2019?**

1. Get all the accident indices from the accidents table for June
2. Filter all casualties for those indices
3. Filter those casualties for ages 0-4

Numpy has the function isin() to select for a list of indices: https://numpy.org/doc/stable/reference/generated/numpy.isin.html

In [ ]:
ind = [x["Accident_Index"] for x in dataraw["accidents"] if "/06/" in x["Date"]]
datafiltered = dataraw["casualties"][np.isin(dataraw["casualties"]["Accident_Index"], ind)]
datafiltered = datafiltered[(datafiltered["Age_of_Casualty"] >= 0) & (datafiltered["Age_of_Casualty"] <= 4)]
len(datafiltered)

**Question: Who killed or injured them?**

Homework

## Association test between two categorical variables 
**(Pearson $\chi^2$ test of independence)**

Inspired by:  
https://peterstatistics.com/CrashCourse/3-TwoVarUnpair/NomNom/NomNom-2a-Test.html  
https://bit.ly/3kbwKEL

**Let us ask: Is there a statistically significant association between accident severity and speed limit?**  
We ask because speed limit is something that the city government can regulate.

### Hypothesis testing

We are now in the realm of [Statistical hypothesis testing](https://en.wikipedia.org/wiki/Statistical_hypothesis_testing). In general, we must first state and compare two hypotheses:

- $H_0$ (null hypothesis): There is no statistically significant relationship between accident severity and speed limit.
- $H_\alpha$ (alternative hypothesis): There is a statistically significant relationship between accident severity and speed limit.

We must then 1) state+check statistical assumptions, 2) choose an appropriate test and test statistic $T$, 3) derive the distribution for the test statistic, 4) select a significance level $\alpha$, usually 0.01 or 0.05, 5) calculate the observed test statistic $t_{\mathrm obs}$, 6) calculate the [p-value](https://en.wikipedia.org/wiki/P-value). 

If the p-value $< \alpha$, then the null hypothesis will be rejected.

### Pearson $\chi^2$ test of independence

To test association between two categorical variables, one uses the [Pearson chi-square test of independence](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test). If the significance of this test (p-value) is below a significance level (typically 0.05), the two variables have a significant association.

The Pearson chi-square test should only be used if most cells have an expected count above 5, and the minimum expected count is at least 1.

In [ ]:
from scipy.stats import chi2_contingency

In [ ]:
ind = (dataraw["accidents"]["Speed_limit"] != -1)
severityspeed = np.array([dataraw["accidents"]["Speed_limit"][ind], dataraw["accidents"]["Accident_Severity"][ind]]).T
print(severityspeed.shape)
print(severityspeed)

We crosstabulate using pandas:
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html

The cross tabulation is also known as contingency table.

In [ ]:
observed_pd = pd.crosstab(severityspeed[:, 0], severityspeed[:, 1], rownames = ["Speed limit"], colnames = ["Accident Severity"]) # To explain expected values, show also with , normalize='index', and , normalize='columns' https://stackoverflow.com/questions/21247203/how-to-make-a-pandas-crosstab-with-percentages
observed = observed_pd.to_numpy()
observed_pd

The idea is now to compare these observed values with expected values.  
The expected values can be calculated using:
\begin{equation*}
E_{i,j} = \frac{R_i \times C_j}{N}
\end{equation*}
The $E_{i,j}$ indicates the expected count in row i, column j. The $R_i$ is the row total of row i, and $C_j$ the column total of column j. The $N$ is the grand total.

In [ ]:
expected = np.zeros(observed.shape, dtype=int)
colTotals = observed.sum(axis=0)
rowTotals = observed.sum(axis=1)
N = rowTotals.sum()

for i in range(observed.shape[0]):
    for j in range(observed.shape[1]):
        expected[i,j] = (rowTotals[i] * colTotals[j]) / N

expected

That was the manual way of doing it. `chi2_contingency()` can do it for us automatically:

In [ ]:
chiVal, pVal, df, expected = chi2_contingency(observed)
chiVal, pVal, df, expected.astype(int)

Because the p-value `pVal`=0.0 < 0.05, the result is significant: There is a significant association between speed and accident severity. 

We now know that the association is significant, but how strong is it?  
Cramer's V, for example, can give an answer: https://en.wikipedia.org/wiki/Cram%C3%A9r%27s_V

The formula is:
\begin{equation*}
V=\sqrt{\frac{\chi^{2} / N}{\min (c-1, r-1)}}
\end{equation*}
where $c$ is the number of columns, $r$ is the number of rows.

In [ ]:
V = np.sqrt( (chiVal/N) / (min(observed.shape)-1) )
V

A V close to 0 implies a weak effect.

Let us visualize this and make a human-readable plot and report below.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 6))
severity_labels = ["Fatal", "Serious", "Slight"]
speed_categories = {20: "20 MPH", 30: "30 MPH", 40: "40 MPH", 50: "50 MPH", 60: "60 MPH", 70: "70 MPH"}
x = np.array(list(speed_categories.keys()))

for i, ax in enumerate(axes[0]):
    ax.plot(x, observed[:,i], 'ro-', label='Observed')
    ax.plot(x, expected[:,i], 'bo-', label='Expected')
    if i==0: 
        ax.set_ylabel('Casualties')
        ax.legend(loc='best');
    ax.set_title(severity_labels[i])
    ax.set_xticks(x)
    ax.set_xticklabels(list(speed_categories.values()))
    fig.autofmt_xdate(rotation=45)

for i, ax in enumerate(axes[1]):
    ax.plot(x, observed[:,i]/expected[:,i], 'go-')
    ax.plot(x, np.ones(x.shape), 'k:')
    
    if i==0: 
        ax.set_ylabel('Observed/Expected')
    ax.set_xticks(x)
    ax.set_xticklabels(list(speed_categories.values()))
    fig.autofmt_xdate(rotation=45)

An association only tells us about correlation, not causation. However, in this case, there is good reason to say that the speed of cars are a cause of the accident severity. Although Cramer's V is low, there is a very clear effect, especially for fatal collisions, $\chi^2$(10, N = 117456) = 1951.03, p < .001, V = 0.091.

The conclusion is therefore that **"Speed kills"**:  
Fatal collisions are over 2 times more likely than expected for high speeds (>60 mph), and are 2-3 times less likely than expected for low speed limits (<30 mph). An urban planning policy recommendation would therefore be: To reduce fatalities, reduce speed limits.

What about statistical tests for different combinations of numerical/categorical variables?
<img src="../references/flowchart-for-choosing-a-statistical-test.png" width="600px"/>

<hr>

# Lecture 4: Spatial filtering

In [ ]:
cityname = "Bristol"
cityid = "bristol"

## Filtering with external table

In [ ]:
lsoa = np.genfromtxt(PATH["data_external"] + "Lower_Layer_Super_Output_Area__December_2011__EW_BSC_V2.csv", delimiter=',', dtype=None, names=True, encoding='utf-8-sig')

In [ ]:
lsoa

Let's select all rows for the city name:

In [ ]:
np.count_nonzero(lsoa['LSOA11NM'] == cityname)

**Oops. We did not find a single row. Why?**

https://stackoverflow.com/questions/38974168/finding-entries-containing-a-substring-in-a-numpy-array

In [ ]:
city_LSOA11CD_rows = np.flatnonzero(np.core.defchararray.find(lsoa['LSOA11NM'],cityname)!=-1)
city_LSOA11CD = lsoa['LSOA11CD'][city_LSOA11CD_rows]
city_LSOA11CD

We want to use this list of LSOA11 codes to restrict our accident data set.

In [ ]:
city_accidentindices = dataraw["accidents"]["Accident_Index"][np.isin(dataraw["accidents"]["LSOA_of_Accident_Location"], city_LSOA11CD)]

Filter

In [ ]:
datacity = {}
for tablename in TABLENAMES:
    datacity[tablename] = dataraw[tablename][np.isin(dataraw[tablename]["Accident_Index"], city_accidentindices)]

In [ ]:
len(datacity["accidents"]), len(datacity["vehicles"]), len(datacity["casualties"])

Export

In [ ]:
for tablename in TABLENAMES:
    with open(PATH["data_interim"] + tablename + "_" + cityid + ".csv", "w") as f:
        w = csv.writer(f)
        w.writerow(dataraw[tablename].dtype.names)
        w.writerows(datacity[tablename])

## Spatial filtering with shapely

<font color="red">Instructor only: Run one time to get JSON</font>

In [ ]:
import json
import shapely
from shapely.geometry import Point, MultiPoint, LineString, MultiLineString, Polygon, MultiPolygon

In [ ]:
with open(PATH["data_external"] + "lad.json") as f:
    jsn = json.load(f)
cities = {"Birmingham": "Birmingham", "Leeds": "Leeds", "Sheffield": "Sheffield", "Bradford": "Bradford", "Liverpool": "Liverpool", "Manchester": "Manchester", "Bristol, City of": "Bristol"}

cities_json = {}
for i in range(len(jsn['features'])):
    if jsn['features'][i]["properties"]["LAD13NM"] in cities:
        cities_json[cities[jsn['features'][i]["properties"]["LAD13NM"]]] = jsn['features'][i]
        print(cities[jsn['features'][i]["properties"]["LAD13NM"]])
with open(PATH["data_processed"] + "citieslad.json", "w") as f:
    json.dump(cities_json, f)

<font color="red">Instructor only END</font>

#### For lecture

In [ ]:
# Add to beginning imports
import json
import shapely
from shapely.geometry import Point, MultiPoint, LineString, MultiLineString, Polygon, MultiPolygon

In [ ]:
with open(PATH["data_processed"] + "citieslad.json") as f:
    cities_json = json.load(f)

In [ ]:
cities_json

In [ ]:
city_boundary = shapely.geometry.shape(cities_json[cityname]["geometry"])
type(city_boundary)

Jupyter visualizes shapely objects!

In [ ]:
city_boundary

In [ ]:
city_boundary.geom_type

Let's get all accident coordinates (from the whole UK)

In [ ]:
lon_list = dataraw["accidents"]["Longitude"]
lat_list = dataraw["accidents"]["Latitude"]

`contains()` and `within()` check for point inclusion:

In [ ]:
print(city_boundary.contains(Point(lon_list[0], lat_list[0])))
print(city_boundary.contains(Point(lon_list[102135], lat_list[102135])))

In [ ]:
city_acc_rowindices = []
for i in range(len(lon_list)):
    if Point(lon_list[i], lat_list[i]).within(city_boundary):
        city_acc_rowindices.append(i)

In [ ]:
len(city_acc_rowindices)

In [ ]:
datacity = {}
datacity["accidents"] = dataraw["accidents"][city_acc_rowindices]
len(datacity["accidents"])

In [ ]:
with open(PATH["data_interim"] + "accidents_" + cityid + "lad.csv", "w") as f:
    w = csv.writer(f)
    w.writerow(data_acc.dtype.names)
    w.writerows(city_data_acc)

Limit vehicles and casualties to these AccidentIndices

In [ ]:
datacity["vehicles"] = dataraw["vehicles"][np.isin(dataraw["vehicles"]["Accident_Index"], datacity["accidents"]["Accident_Index"])]
datacity["casualties"] = dataraw["casualties"][np.isin(dataraw["casualties"]["Accident_Index"], datacity["accidents"]["Accident_Index"])]

In [ ]:
len(datacity["accidents"]), len(datacity["vehicles"]), len(datacity["casualties"])

In [ ]:
!diff ../data/interim/accidents_bristol.csv ../data/interim/accidents_bristol_lad.csv

What does it mean?  
See: https://www.computerhope.com/unix/udiff.htm

<hr>

# Lecture 5: Visualizing spatial data

In [ ]:
# Add these to the imports in the beginning
import folium
from folium import plugins
from folium.plugins import HeatMap, MarkerCluster

Inspired by: https://alysivji.github.io/getting-started-with-folium.html

In [ ]:
latlons = np.vstack((datacity["accidents"]['Latitude'], datacity["accidents"]['Longitude'])).T
centroid = list(MultiPoint(latlons).centroid.coords)[0]
m1 = folium.Map(centroid, zoom_start=11)
for row in datacity["accidents"]:
    folium.CircleMarker([row['Latitude'], row['Longitude']],
                        radius = 5,
                        popup = row['Accident_Index'] + "\n" + row["Date"] + ", " + row["Time"],
                        fill_color = "#3db7e4",
                       ).add_to(m1)

HeatMap(latlons).add_to(folium.FeatureGroup(name='Heat Map').add_to(m1))
folium.LayerControl().add_to(m1)
m1

The heatmap is built with KDE:  
https://en.wikipedia.org/wiki/Kernel_density_estimation

We can also add automatic clusters:

In [ ]:
latlons = np.vstack( (datacity["accidents"]['Latitude'], datacity["accidents"]['Longitude'])).T
centroid = list(MultiPoint(latlons).centroid.coords)[0]
m2 = folium.Map(centroid, zoom_start=11)
marker_cluster = MarkerCluster().add_to(folium.FeatureGroup(name='Clusters').add_to(m2))
for row in datacity["accidents"]:
    folium.CircleMarker([row['Latitude'], row['Longitude']],
                        radius = 5,
                        popup = row['Accident_Index'] + "\n" + row["Date"] + ", " + row["Time"],
                        fill_color = "#3db7e4",
                       ).add_to(marker_cluster)

HeatMap(latlons).add_to(folium.FeatureGroup(name='Heat Map').add_to(m2))
folium.LayerControl().add_to(m2)
m2